<a href="https://colab.research.google.com/github/taechanha/inverseDesignMetaMatl/blob/main/%5Br%5D_vae_adjmat_brainnet_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###################### FROM HERE ######################
###################### FROM HERE ######################
###################### FROM HERE ######################
###################### FROM HERE ######################
###################### FROM HERE ######################
###################### FROM HERE ######################
###################### FROM HERE ######################
###################### FROM HERE ######################

import numpy as np
import matplotlib.pyplot as plt

f = open("/content/dset4109.txt", 'r')
data = []
length = 0
for i in f:
  new = []
  new.append(i)
  data.append(new)

  # length of dataset
  length += 1

f.close()

# create dataset from data
dataset = []
for i in range(length):
  new = []
  for j in data[i][0].split(','):
    new.append(float(j))
  dataset.append(new)


# trim out label from dataset
# Ex Ey Ez
label = []
new = []
for line in dataset:
  tmp = []
  tmp.append(line.pop(-1))
  tmp.append(line.pop(-1))
  tmp.append(line.pop(-1))
  tmp.reverse()
  label.append(tmp)

print("label.shape: ", np.array(label).shape)


# create edge_index
edge_mat = np.zeros(shape=(27,27))
edge_index = []

for e in range(len(dataset)):
  for i in range(0, 27):
    for j in range(i+1, 27):
      edge_mat[i][j] = dataset[e].pop(0)
  edge_index.append(edge_mat + edge_mat.T)


# whole dataset to Tensor & train/test split
import os
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

edge_index = torch.FloatTensor(edge_index)
label = torch.FloatTensor(label)

split = int(length * 0.8)

x_train = edge_index[:split]
y_train = label[:split]
x_val = edge_index[split:]
y_val = label[split:]

print("x_train, y_train, x_val shape: ", x_train.shape, y_train.shape, x_val.shape)

# create torch dataset
import os
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

class CustomDataset(Dataset):
    def __init__(self, label, dataset, transform=None, target_transform=None):
        self.labels = label
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.labels[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

# create DataLoader
from torch.utils.data import DataLoader

training_data   = CustomDataset(label=y_train, dataset=x_train)
test_data       = CustomDataset(label=y_val, dataset=x_val)
train_loader    = DataLoader(training_data, batch_size=4, shuffle=True)
test_loader     = DataLoader(test_data, batch_size=4, shuffle=False)

label.shape:  (4109, 3)
x_train, y_train, x_val shape:  torch.Size([3287, 27, 27]) torch.Size([3287, 3]) torch.Size([822, 27, 27])


In [1]:
class E2EBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, bias=False):
        super(E2EBlock, self).__init__()
        self.cnn1 = torch.nn.Conv2d(in_dim, out_dim, (1, 27), bias=bias)
        self.cnn2 = torch.nn.Conv2d(in_dim, out_dim, (27, 1), bias=bias)

        #nn.init.xavier_uniform_(self.cnn2.weight)

    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)

        return torch.cat([a]*27, 3) + torch.cat([b]*27, 2)

class BrainNetVAE(torch.nn.Module):
    def __init__(self):
        super(BrainNetCNN, self).__init__()
        
        self.e2econv1 = E2EBlock(1, 8, bias=True)  #self.conv5_bn = nn.BatchNorm2d(256)

        self.E2N = nn.Conv2d(8, 1, (1, 27))
        self.N2G = nn.Conv2d(1, 32, (27, 1))

        self.fc11 = nn.Linear(32, 2)
        self.fc12 = nn.Linear(32, 2)


        self.rfc1 = nn.Linear(3, 32)
        self.rN2G = nn.Conv2d(32, 1, (27, 1))
        self.rE2N = nn.Conv2d(1, 8, (1, 27))

        self.re2econv1 = E2EBlock(8, 1, bias=False)


    def encode(self, x):
        print(x.shape)
        x = F.relu(self.e2econv1(x.unsqueeze(1)))
        print(x.shape)
        x = F.relu(self.E2N(x))
        x = F.relu(self.N2G(x))
        print(x.shape)
        x = x.view(x.size(0), -1)
        print(x.shape)
        return self.fc11(x), self.fc12(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
        
    def decode(self, x):
        x = F.relu(self.rfc1(x))
        x = F.relu(self.rN2G(x))
        x = F.relu(self.rE2N(x))

        x = self.re2conv1(x)
        return x

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 729))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data, nn.init.calculate_gain('relu'))
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, nn.init.calculate_gain('relu'))

torch.manual_seed(42)
model = BrainNetVAE().cuda()
model.apply(weights_init)
print(model)

NameError: ignored

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1  = nn.Linear(351, 128)
        
        self.fc51  = nn.Linear(128, 10)
        self.fc52  = nn.Linear(128, 10)

        self.fc5  = nn.Linear(10, 128)
        self.fc6  = nn.Linear(128, 351)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        return self.fc51(x), self.fc52(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        x = F.relu(self.fc5(z))
        x = self.fc6(x)
        return torch.sigmoid(x)

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 351))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar